In [2]:
!pip install ktrain

     |████████████████████████████████| 174kB 3.5MB/s 
     |████████████████████████████████| 317kB 52.7MB/s 
     |████████████████████████████████| 112kB 47.9MB/s 
     |████████████████████████████████| 1.0MB 38.3MB/s 
     |████████████████████████████████| 204kB 43.5MB/s 
     |████████████████████████████████| 1.8MB 43.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.6.0-cp36-none-any.whl size=223615 sha256=8da8c42f7f4b1b2612d6829bea1e9ca724685e38d15b68a7bab97adc54f8ccc7
  Stored in directory: /root/.cache/pip/wheels/67/d5/4e/5ff9bfbef0091828dc1f68f1fb4191df900fa8063d9599d5bc
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=b59ce52a5f2f9969a4b4e96be5879177a33bc44534178644dd11ea2a9155cb89
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=bc333214e11b6c6570bc5b912c7a3a23f30d6ae296f9960c87f

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
df=pd.read_excel('/content/Data_Train.xlsx')

In [5]:
df.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [0]:
dummy=pd.get_dummies(df["SECTION"])

In [0]:
df=pd.concat([df,dummy],axis=1)

In [8]:
df.head()

,STORY,SECTION,0,1,2,3
0,But the most painful was the huge reversal in ...,3,0,0,0,1
1,How formidable is the opposition alliance amon...,0,1,0,0,0
2,Most Asian currencies were trading lower today...,3,0,0,0,1
3,"If you want to answer any question, click on ‘...",1,0,1,0,0
4,"In global markets, gold prices edged up today ...",3,0,0,0,1


In [0]:
df=df.drop('SECTION',axis=1)

In [10]:
df.head()

,STORY,0,1,2,3
0,But the most painful was the huge reversal in ...,0,0,0,1
1,How formidable is the opposition alliance amon...,1,0,0,0
2,Most Asian currencies were trading lower today...,0,0,0,1
3,"If you want to answer any question, click on ‘...",0,1,0,0
4,"In global markets, gold prices edged up today ...",0,0,0,1


In [11]:
import ktrain
from ktrain import text


Using TensorFlow backend.


using Keras version: 2.2.4


In [0]:
df.to_csv('main.csv')

In [13]:
DATA_PATH = '/content/main.csv'
NUM_WORDS = 500000
MAXLEN = 150
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv(DATA_PATH,
                      'STORY',
                      label_columns = ["0", "1", "2", "3"],
                      val_filepath=None, # if None, 10% of data will be used for validation
                      max_features=NUM_WORDS, maxlen=MAXLEN,
                      ngram_range=1)

detected encoding: utf-8 (if wrong, set manually)
language: en
Word Counts: 36515
Nrows: 6865
6865 train sequences
Average train sequence length: 109
x_train shape: (6865,150)
y_train shape: (6865,4)
763 test sequences
Average test sequence length: 108
x_test shape: (763,150)
y_test shape: (763,4)


In [14]:
model = text.text_classifier('bigru', (x_train, y_train), preproc=preproc,multilabel=True)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
processing pretrained word vectors...
Loading pretrained word vectors...this may take a few moments...
downloading pretrained word vectors (~1.5G) ...
[██████████████████████████████████████████████████]
extracting pretrained pretrained word vectors...
done.

cleanup downloaded zip...
done.

Done.
done.


In [0]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test))

In [0]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
RocAuc = RocAucEvaluation(validation_data=(x_test, y_test), interval=1)

In [17]:
learner.autofit(0.001, 3, callbacks=[RocAuc])



begin training using triangular learning rate policy with max lr of 0.001...


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:421: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 150000000 elements. This may consume a large amount of memory.
  num_elements)


Train on 6865 samples, validate on 763 samples
Epoch 1/3
6865/6865 [==============================] - 457s 67ms/step - loss: 0.2611 - acc: 0.8903 - val_loss: 0.0769 - val_acc: 0.9725

 ROC-AUC - epoch: 1 - score: 0.996414 

Epoch 2/3
6865/6865 [==============================] - 456s 66ms/step - loss: 0.0567 - acc: 0.9826 - val_loss: 0.0532 - val_acc: 0.9800

 ROC-AUC - epoch: 2 - score: 0.997628 

Epoch 3/3
6865/6865 [==============================] - 460s 67ms/step - loss: 0.0363 - acc: 0.9891 - val_loss: 0.0499 - val_acc: 0.9823

 ROC-AUC - epoch: 3 - score: 0.998111 



In [18]:
!pip install h5py

In [0]:
learner.save_model('model.h5')

In [0]:
from numpy import loadtxt
from keras.models import load_model

In [0]:
import sklearn

In [0]:
test=pd.read_excel('/content/Data_Test.xlsx')

In [0]:
p = ktrain.get_predictor(learner.model, preproc)

In [71]:
p.predict('Ripple also helps bank customers send money to people in many emerging markets including Mexico, India, and Thailand to increase their share of “this large and growing market". What’s next? “Ripple is moving beyond blockchain, and connecting networks so that we can move money across networks. Again this is open-source and lightweight so it becomes easy to transfer money across networks. So we are building the ecosystem for networks to connect with each other and in our view globalization will be completed when data, goods and money flow seamlessly. That’s the way we think of it as an internet of value when the whole world gets connected through payment systems," Gupta said.')

[('0', 0.0040331194),
 ('1', 0.99136996),
 ('2', 0.0007945218),
 ('3', 0.0028030227)]

In [0]:
result= p.predict([i for i in test['STORY']])

In [43]:
result.count

<function list.count>

In [23]:
result[0]

[('0', 0.0012842417),
 ('1', 0.9999567),
 ('2', 0.00028780103),
 ('3', 0.00017616153)]

In [0]:
submit=[]

In [0]:
for i in range(0,2748):
  lst = result[i]
  submit.append(max(lst,key=lambda x:x[1])[0])

In [75]:
submit[0]



'1'

In [0]:
df = pd.DataFrame({'ImageId': test.STORY , 'Label': submit})
df.to_excel('submit.xlsx', index = False)


In [45]:
learner.validate(val_data=(x_test, y_test))

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:118: UserWarning: multilabel confusion matrices not yet supported
  warnings.warn('multilabel confusion matrices not yet supported')


In [50]:
learner.validate()

/usr/local/lib/python3.6/dist-packages/ktrain/core.py:118: UserWarning: multilabel confusion matrices not yet supported
  warnings.warn('multilabel confusion matrices not yet supported')
